<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/app/gradioapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

In [1]:
import gradio as gr

In [ ]:
#@title Hello wolrd examples

In [ ]:
def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch()

In [ ]:
def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(
    fn=greet,
    inputs=gr.Textbox(lines=2, placeholder="Name Here Test..."),
    outputs="text",
)
demo.launch()

In [2]:
def greet(name, is_morning, temperature):
    salutation = "Good morning" if is_morning else "Good evening"
    greeting = f"{salutation} {name}. It is {temperature} degrees today"
    celsius = (temperature - 32) * 5 / 9
    return greeting, round(celsius, 2)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "checkbox", gr.Slider(0, 100)],
    outputs=["text", "number"],
)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [3]:
#@title Image examples
import numpy as np
import gradio as gr

def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189], 
        [0.349, 0.686, 0.168], 
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

demo = gr.Interface(sepia, gr.Image(shape=(200, 200)), "image")
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
#@title chatbot app

In [ ]:
#style.css
# create style.css
"""
col-container {max-width: 700px; margin-left: auto; margin-right: auto;}

a {text-decoration-line: underline; font-weight: 600;}
.footer {
        margin-bottom: 45px;
        margin-top: 10px;
        text-align: center;
        border-bottom: 1px solid #e5e5e5;
    }
    .footer>p {
        font-size: .8rem;
        display: inline-block;
        padding: 0 10px;
        transform: translateY(10px);
        background: white;
    }
    .dark .footer {
        border-color: #303030;
    }
    .dark .footer>p {
        background: #0b0f19;
    }
.animate-spin {
    animation: spin 1s linear infinite;
}
@keyframes spin {
    from {
        transform: rotate(0deg);
    }
    to {
        transform: rotate(360deg);
    }
}
#upload_button {
  background-color: black;
  color: white;
}
"""

In [ ]:
!pip install gradio
!pip install -qq git+https://github.com/huggingface/diffusers.git

In [ ]:
!pip install transformers accelerate

In [2]:
!pip install xformers triton

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 71.6 MB/s eta 0:00:00


In [3]:
import PIL
import requests
import torch
import gradio as gr
import random
from PIL import Image
import os
import time
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler

In [4]:
#Loading from Diffusers Library
model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16") #, safety_checker=None)
pipe.to("cuda")
#pipe.enable_attention_slicing()
pipe.enable_xformers_memory_efficient_attention()
pipe.unet.to(memory_format=torch.channels_last)


help_text = """ 
**Note: Please be advised that a safety checker has been implemented in this public space. 
    Any attempts to generate inappropriate or NSFW images will result in the display of a black screen 
    as a precautionary measure to protect all users. We appreciate your cooperation in 
    maintaining a safe and appropriate environment for all members of our community.**
    
    New features and bug-fixes: 
    
    1. Chat style interface
    2. Now use **'reverse'** as prompt to get back the previous image after an unwanted edit
    3. Use **'restart'** as prompt to get back to original image and start over!
    4. Now you can load larger image files (~5 mb) as well
Some notes from the official [instruct-pix2pix](https://huggingface.co/spaces/timbrooks/instruct-pix2pix) Space by the authors and from the official [Diffusers docs](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/pix2pix) -
If you're not getting what you want, there may be a few reasons:
1. Is the image not changing enough? Your guidance_scale may be too low. It should be >1. Higher guidance scale encourages to generate images 
that are closely linked to the text `prompt`, usually at the expense of lower image quality. This value dictates how similar the output should 
be to the input. This pipeline requires a value of at least `1`. It's possible your edit requires larger changes from the original image. 
                
2. Alternatively, you can toggle image_guidance_scale. Image guidance scale is to push the generated image towards the inital image. Image guidance 
                scale is enabled by setting `image_guidance_scale > 1`. Higher image guidance scale encourages to generate images that are closely 
                linked to the source image `image`, usually at the expense of lower image quality.  
3. I have observed that rephrasing the instruction sometimes improves results (e.g., "turn him into a dog" vs. "make him a dog" vs. "as a dog").
4. Increasing the number of steps sometimes improves results.
5. Do faces look weird? The Stable Diffusion autoencoder has a hard time with faces that are small in the image. Try:
    * Cropping the image so the face takes up a larger portion of the frame.
"""

css = """
#col-container {max-width: 580px; margin-left: auto; margin-right: auto;}
a {text-decoration-line: underline; font-weight: 600;}
.footer {
        margin-bottom: 45px;
        margin-top: 10px;
        text-align: center;
        border-bottom: 1px solid #e5e5e5;
    }
    .footer>p {
        font-size: .8rem;
        display: inline-block;
        padding: 0 10px;
        transform: translateY(10px);
        background: white;
    }
    .dark .footer {
        border-color: #303030;
    }
    .dark .footer>p {
        background: #0b0f19;
    }
.animate-spin {
    animation: spin 1s linear infinite;
}
@keyframes spin {
    from {
        transform: rotate(0deg);
    }
    to {
        transform: rotate(360deg);
    }
}
"""

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [17]:
def previous(image):
    return image 

def upload_image(file):
    return Image.open(file)

def upload_button_config():
    return gr.update(visible=False)

def upload_textbox_config(text_in):
    return gr.update(visible=True)

In [ ]:
def chat(btn_upload, image_in, in_steps, in_guidance_scale, in_img_guidance_scale, image_hid, img_name, counter_out, image_oneup, prompt, history, progress=gr.Progress(track_tqdm=True)):
    progress(0, desc="Starting...")
    if prompt != '' and prompt.lower() == 'reverse' : #--to add revert functionality later
        history = history or []
        temp_img_name = img_name[:-4]+str(int(time.time()))+'.png' 
        image_oneup.save(temp_img_name)
        response = 'Reverted to the last image ' + '<img src="/file=' + temp_img_name + '">'  
        history.append((prompt, response))
        return history, history, image_oneup, temp_img_name, counter_out
    if prompt != '' and prompt.lower() == 'restart' : #--to add revert functionality later
        history = history or []
        temp_img_name = img_name[:-4]+str(int(time.time()))+'.png' 
        #Resizing the image
        basewidth = 512
        wpercent = (basewidth/float(image_in.size[0]))
        hsize = int((float(image_in.size[1])*float(wpercent)))
        image_in = image_in.resize((basewidth,hsize), Image.Resampling.LANCZOS)
        image_in.save(temp_img_name)
        response = 'Reverted to the last image ' + '<img src="/file=' + temp_img_name + '">'  
        history.append((prompt, response))
        return history, history, image_in, temp_img_name, counter_out
    #adding supportive sample text
    add_text_list = ["There you go", "Enjoy your image!", "Nice work! Wonder what you gonna do next!", "Way to go!", "Does this work for you?", "Something like this?"]        
    if counter_out == 0:
      t1 = time.time()
      print(f"Time at start = {t1}")
      #convert file object to image
      image_in = Image.open(btn_upload)
      
      #Resizing the image
      basewidth = 512
      wpercent = (basewidth/float(image_in.size[0]))
      hsize = int((float(image_in.size[1])*float(wpercent)))
      image_in = image_in.resize((basewidth,hsize), Image.Resampling.LANCZOS)
            
      # Save the image to the file-like object
      seed = random.randint(0, 1000000)
      img_name = f"./edited_image_{seed}.png"
      image_in.save(img_name)
      
      #add state
      history = history or []
      response = '<img src="/file=' + img_name + '">'
      history.append((prompt, response))
      counter_out += 1
      
      t2 = time.time()
      print(f"Time at end = {t2}")
      time_diff = t2-t1
      print(f"Time taken = {time_diff}")
      return history, history, image_in, img_name, counter_out
    
    elif counter_out == 1:        
      #instruct-pix2pix inference
      edited_image = pipe(prompt, image=image_in, num_inference_steps=int(in_steps), guidance_scale=float(in_guidance_scale), image_guidance_scale=float(in_img_guidance_scale)).images[0]
      if os.path.exists(img_name):
        os.remove(img_name)
      temp_img_name = img_name[:-4]+str(int(time.time()))[-4:] +'.png' 
      with open(temp_img_name, "wb") as fp:
        # Save the image to the file-like object
        edited_image.save(fp)
      #Get the name of the saved image
      saved_image_name1 = fp.name
      history = history or []
      response = random.choice(add_text_list) + '<img src="/file=' + saved_image_name1 + '">'   #IMG_NAME
      history.append((prompt, response))
      counter_out += 1
      return history, history, edited_image, temp_img_name, counter_out
    
    elif counter_out > 1:
      edited_image = pipe(prompt, image=image_hid, num_inference_steps=int(in_steps), guidance_scale=float(in_guidance_scale), image_guidance_scale=float(in_img_guidance_scale)).images[0]
      if os.path.exists(img_name):
        os.remove(img_name)
      temp_img_name = img_name[:-4]+str(int(time.time()))[-4:]+'.png' 
      # Create a file-like object
      with open(temp_img_name, "wb") as fp:
        # Save the image to the file-like object
        edited_image.save(fp)
      #Get the name of the saved image
      saved_image_name2 = fp.name
      #edited_image.save(temp_img_name) #, overwrite=True)
      history = history or []
      response = random.choice(add_text_list) + '<img src="/file=' + saved_image_name2 + '">'  
      history.append((prompt, response))
      counter_out += 1
      return history, history, edited_image, temp_img_name, counter_out

In [15]:
#Blocks layout
with gr.Blocks(css="style.css") as demo:
    with gr.Column(elem_id="col-container"):
        gr.HTML("""<div style="text-align: center; max-width: 700px; margin: 0 auto;">
                <div
                style="
                    display: inline-flex;
                    align-items: center;
                    gap: 0.8rem;
                    font-size: 1.75rem;
                "
                >
                <h1 style="font-weight: 900; margin-bottom: 7px; margin-top: 5px;">
                    ChatPix2Pix: Image Editing by Instructions
                </h1>
                </div>
            </div>""")
        with gr.Accordion("Advance settings for Training and Inference", open=False):
          image_in = gr.Image(visible=False,type='pil', label="Original Image")
          gr.Markdown("Advance settings for - Number of Inference steps, Guidanace scale, and Image guidance scale.")
          in_steps = gr.Number(label="Enter the number of Inference steps", value = 20)
          in_guidance_scale = gr.Slider(1,10, step=0.5, label="Set Guidance scale", value=7.5)
          in_img_guidance_scale = gr.Slider(1,10, step=0.5, label="Set Image Guidance scale", value=1.5)
          image_hid = gr.Image(type='pil', visible=False)
          image_oneup = gr.Image(type='pil', visible=False)
          img_name_temp_out = gr.Textbox(visible=False)
          counter_out = gr.Number(visible=False, value=0, precision=0)
            
        #with gr.Row():
        text_in = gr.Textbox(value='', placeholder="Type your instructions here and press enter", elem_id = "input_prompt", visible=False, label='Great! Now you can edit your image with Instructions')
        btn_upload = gr.UploadButton("Upload image to start editing", file_types=["image"], file_count="single", elem_id="upload_button")
        
        # chatbot = gr.Chatbot(elem_id = 'chatbot-component', label='Conversational editing for Images')
        state_in = gr.State()

    """
    #Using Event Listeners
    btn_upload.upload(chat,
                      [btn_upload, image_in, in_steps, in_guidance_scale, in_img_guidance_scale, image_hid, img_name_temp_out,counter_out, image_oneup,  text_in, state_in], 
                      [chatbot, state_in, image_in, img_name_temp_out, counter_out])
    btn_upload.upload(fn = upload_textbox_config, inputs=text_in, outputs = text_in)

    text_in.submit(chat,[btn_upload, image_in, in_steps, in_guidance_scale, in_img_guidance_scale, image_hid, img_name_temp_out,counter_out, image_oneup,  text_in, state_in], [chatbot, state_in, image_hid, img_name_temp_out, counter_out])
    text_in.submit(previous, [image_hid], [image_oneup])
    
    chatbot.change(fn = upload_button_config, outputs=btn_upload) #, scroll_to_output = True)
    text_in.submit(None, [], [], _js = "() => document.getElementById('#chatbot-component').scrollTop = document.getElementById('#chatbot-component').scrollHeight")
    """
    #with gr.Accordion("Release Notes", open=False):
    gr.Markdown(help_text)

In [16]:
demo.queue(concurrency_count=10)
demo.launch(debug=True, width="80%", height=2000)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e879600c-cbf5-4bb8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e879600c-cbf5-4bb8.gradio.live


In [ ]:
# yolov8 apps

In [ ]:
!pip install ultralytics gradio

In [13]:
from ultralytics import YOLO
yolo_model = "yolov8l.pt"
model = YOLO(yolo_model)
img_path = "/content/zidane.jpeg"
infer_size = 640
conf = 0.6
iou = 0.5
results = model(source=img_path, imgsz=infer_size, conf=conf, iou=iou)
results = list(results)[0]

Ultralytics YOLOv8.0.29 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
YOLOv8l summary (fused): 268 layers, 43668288 parameters, 0 gradients, 165.2 GFLOPs

image 1/1 /content/zidane.jpeg: 384x640 2 persons, 1 tie, 1851.9ms
Speed: 4.6ms pre-process, 1851.9ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
results

In [18]:
xyxy_list = results.boxes.xyxy.cpu().numpy().tolist()
conf_list = results.boxes.conf.cpu().numpy().tolist()
cls_list = results.boxes.cls.cpu().numpy().tolist()

In [1]:
import csv
import sys
from pathlib import Path
import yaml
import random
from ultralytics import YOLO

In [ ]:
# model_config/model_name_p5_all.yaml
# model_names: ["yolov8n", "yolov8s", "yolov8m", "yolov8l", "yolov8x", "yolov8n-seg", "yolov8s-seg", "yolov8m-seg", "yolov8l-seg", "yolov8x-seg"]

In [2]:
# cls_name/cls_name_en.yaml
"""
model_cls_name: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant',
    'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
    'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle',
    'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli',
    'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet',
    'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]
"""

"\nmodel_cls_name: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',\n    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant',\n    'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',\n    'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle',\n    'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli',\n    'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet',\n    'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator',\n    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'\n]\n"

In [3]:
suffix_list = [".csv", ".yaml"]

In [4]:
# yaml文件解析
def yaml_parse(file_path):
    return yaml.safe_load(open(file_path, encoding="utf-8").read())

# yaml csv
def yaml_csv(file_path, file_tag):
    file_suffix = Path(file_path).suffix
    if file_suffix == suffix_list[0]:
        file_names = [i[0] for i in list(csv.reader(open(file_path)))]  # csv版
    elif file_suffix == suffix_list[1]:
        file_names = yaml_parse(file_path).get(file_tag)  # yaml版
    else:
        print(f"{file_path}")
        sys.exit()

    return file_names

In [5]:
model_cfg = "/content/model_config/model_name_p5_all.yaml"
cls_name = "/content/cls_name/cls_name_en.yaml"

model_names = yaml_csv(model_cfg, "model_names")  # 模型名称
model_cls_name = yaml_csv(cls_name, "model_cls_name")  # 类别名称

model_cls_name_cp = model_cls_name.copy()  # 类别名称

In [ ]:
model_cls_name_cp

In [7]:
def random_color(cls_num, is_light=True):
    color_list = []
    for i in range(cls_num):
        color = (
            random.randint(0, 127) + int(is_light) * 128,
            random.randint(0, 127) + int(is_light) * 128,
            random.randint(0, 127) + int(is_light) * 128,
        )
        color_list.append(color)

    return color_list


color_list = random_color(len(model_cls_name_cp), True)

In [18]:
from PIL import Image, ImageDraw
import cv2
import numpy as np

def draw_image_with_bbox(img, score_list, bbox_list, cls_list, cls_index_list, color_list):
  img_pil = ImageDraw.Draw(img)
  for score, (xmin, ymin, xmax, ymax), label, cls_index in zip(score_list, bbox_list, cls_list, cls_index_list):
        img_pil.rectangle([xmin, ymin, xmax, ymax], fill=None, outline=color_list[cls_index], width=2)  # 边界框
        countdown_msg = f"{id}-{label} {score:.2f}"

        text_w, text_h = 5, 5

        img_pil.rectangle(
            (xmin, ymin, xmin + text_w, ymin + text_h),
            fill=color_list[cls_index],
            outline=color_list[cls_index],
        )
        img_pil.multiline_text(
            (xmin, ymin),
            countdown_msg,
            fill=(0, 0, 0),
            align="center",
        )

        id += 1
  return img

def polygon_drawing(img_mask, canvas, color_seg):
    color_seg = list(color_seg)
    color_seg[0], color_seg[2] = color_seg[2], color_seg[0]
    color_seg = tuple(color_seg)
    pts = np.array(img_mask, dtype=np.int32)

    cv2.drawContours(canvas, [pts], -1, color_seg, thickness=-1)


def seg_output(img_path, seg_mask_list, color_list, cls_list):
    img = cv2.imread(img_path)
    img_c = img.copy()

    w, h = img.shape[1], img.shape[0]

    for seg_mask, cls_index in zip(seg_mask_list, cls_list):
        img_mask = []
        for i in range(len(seg_mask)):
            img_mask.append([seg_mask[i][0] * w, seg_mask[i][1] * h])

        polygon_drawing(img_mask, img_c, color_list[int(cls_index)])  

    img_mask_merge = cv2.addWeighted(img, 0.3, img_c, 0.7, 0)  

    return img_mask_merge

def det_image(img_path, model_name, infer_size,  conf, iou):
  color_list = random_color(len(model_cls_name_cp), True)

  model = YOLO(model_name + ".pt")
  results = model(source=img_path, imgsz=infer_size, conf=conf, iou=iou)
  results = list(results)[0]

  if (model_name[-3:] == "seg"):
        masks_list = results.masks.segments
        img_mask_merge = seg_output(img_path, masks_list, color_list, cls_list)
        img = Image.fromarray(cv2.cvtColor(img_mask_merge, cv2.COLOR_BGRA2RGBA))
  else:
        img = Image.open(img_path)
  
  det_img = img
  objSize_dict = {}
  clsRatio_dict = {}
  return det_img, objSize_dict, clsRatio_dict

In [9]:
import gradio as gr

In [19]:
source = "upload"
img_tool = "editor"
model_name = "yolov8s"
inference_size=640
slider_step=0.1
nms_conf = 0.5
nms_iou = 0.5
# input parameters
inputs_img = gr.Image(image_mode="RGB", source=source, tool=img_tool, type="filepath", label="base image")
inputs_model01 = gr.Dropdown(choices=model_names, value=model_name, type="value", label="model")
inputs_size01 = gr.Slider(384, 1536, step=128, value=inference_size, label="infer size")
input_conf01 = gr.Slider(0, 1, step=slider_step, value=nms_conf, label="confidence")
inputs_iou01 = gr.Slider(0, 1, step=slider_step, value=nms_iou, label="IoU threshold")

# inputs parameters list
inputs_img_list = [
        inputs_img,  # input image
        inputs_model01,  # model
        inputs_size01,  # inference_size
        input_conf01,  # confidence
        inputs_iou01,  # IoU threshold
]

outputs_img = gr.Image(type="pil", label="output")
outputs_objSize = gr.Label(label="output_size")
outputs_clsSize = gr.Label(label="class_size")

outputs_img_list = [outputs_img, outputs_objSize, outputs_clsSize]

title = "Yolov8 detection"

gyd_img = gr.Interface(
        fn=det_image,
        inputs=inputs_img_list,
        outputs=outputs_img_list,
        title=title,
        # description=description,
        # examples=examples_imgs,
        cache_examples=False,
        flagging_dir="run", 
        # allow_flagging="manual",
        # flagging_options=["good", "generally", "bad"],
)

gyd_img.launch(
        inbrowser=True,  
        show_tips=True, 
        debug=True,
        # share=is_share,  
        # favicon_path="./icon/logo.ico",  
        # show_error=True,  
        # quiet=True,  
)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Tip: Let users specify why they flagged input with the `flagging_options=` kwarg; for example: `gr.Interface(..., flagging_options=['too slow', 'incorrect output', 'other'])`


Ultralytics YOLOv8.0.29 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
YOLOv8l-seg summary (fused): 295 layers, 45973568 parameters, 0 gradients, 220.5 GFLOPs

image 1/1 /tmp/tmpykknfuvj.png: 384x640 1 car, 1 dog, 2209.3ms
Speed: 0.9ms pre-process, 2209.3ms inference, 3.8ms postprocess per image at shape (1, 3, 640, 640)
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/gradio/routes.py", line 344, in run_predict
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.8/dist-packages/gradio/blocks.py", line 1012, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.8/dist-packages/gradio/blocks.py", line 830, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.8/dist-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.8/dist-packages/anyio/_backends/_asyncio.py", line 937, in

Keyboard interruption in main thread... closing server.


[image, dropdown, slider, slider, slider]